1. Turn the dataset into a dataset compatible with Fastext (see the Tips on using FastText section a bit lower)

In [25]:
from datasets import load_dataset

train_dataset = load_dataset("imdb", split="train").to_pandas()
test_dataset = load_dataset("imdb", split="test").to_pandas()

test_dataset

Reusing dataset imdb (C:\Users\matth\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (C:\Users\matth\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [2]:
train_dataset['text'] = train_dataset['text'].str.lower()
test_dataset['text'] = test_dataset['text'].str.lower()

In [3]:
from string import punctuation

train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')

C:\Users\matth\AppData\Local\Temp\ipykernel_12740\1553221715.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_dataset['text'] = train_dataset['text'].str.replace('[{}]'.format(punctuation), '')
C:\Users\matth\AppData\Local\Temp\ipykernel_12740\1553221715.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_dataset['text'] = test_dataset['text'].str.replace('[{}]'.format(punctuation), '')


In [4]:
test_dataset

,text,label
0,i love scifi and am willing to put up with a l...,0
1,worth the entertainment value of a rental espe...,0
2,its a totally average film with a few semialri...,0
3,star rating saturday night friday night fri...,0
4,first off let me say if you havent enjoyed a v...,0
...,...,...
24995,just got around to seeing monster man yesterda...,1
24996,i got this as part of a competition prize i wa...,1
24997,i got monster man in a box set of three films ...,1
24998,five minutes in i started to feel how naff thi...,1


In [33]:
def panda_to_fasttext(file, dataframe):
    with open(file, 'w') as f:
        for index, row in dataframe.iterrows():
            f.writelines(f'__label__{str(row["label"]).encode("utf-8")} {row["text"].encode("utf-8")}\n')

In [34]:
panda_to_fasttext('imdb_train_comments.txt', train_dataset)

In [35]:
panda_to_fasttext('imdb_test_comments.txt', test_dataset)

In [6]:
!pip install fasttext

     ---------------------------------------- 68.8/68.8 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-win_amd64.whl size=243528 sha256=be409ee3bebb374c7b5f95160cb006ef053c70d9032fd134c484dfd27986cb4f
  Stored in directory: c:\users\matth\appdata\local\pip\cache\wheels\64\57\bc\1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


2. Train a FastText classifier with default parameters on the training data, and evaluate it on the test data using accuracy.

In [23]:
import fasttext

model = fasttext.train_supervised('imdb_train_comments.txt')

In [24]:
model.test('imdb_test_comments.txt')  

(25000, 0.84992, 0.84992)

3. Use the hyperparameters search functionality of FastText and repeat step 2

In [29]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(train_dataset, test_size=0.2, random_state=42, shuffle=True)

In [36]:
panda_to_fasttext('imdb_validation_comments.txt', validation)
panda_to_fasttext('imdb_train_comments.txt', train)

In [37]:
model = fasttext.train_supervised(input='imdb_train_comments.txt', autotuneValidationFile='imdb_validation_comments.txt')

In [38]:
model.test('imdb_test_comments.txt')  

(25000, 0.88064, 0.88064)

4. Look at the differences between the default model and the attributes found with hyperparameters search. How do the two models differ?

The second model with parameters has a better accuracy than the first one.

5. Using the tuned model, take at least 2 wrongly classified examples from the test set, and try explaining why the model failed.